In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow.keras as keras
from keras.preprocessing import image



In [ ]:
size = (224,224)
BATCH_SIZE = 32
EPOCHS =30
FILE_PATH = '/content/drive/MyDrive/HACKMOL/adult Content/test_images_adult'

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    FILE_PATH,
    image_size = size,
    shuffle =True ,
    batch_size = BATCH_SIZE
)

Found 1419 files belonging to 2 classes.


In [ ]:
dataset.class_names

['Adult', 'normal']

In [ ]:
class_names = {"Adult" :0 , "normal" :1}

In [ ]:
for image_batch , labels in dataset.take(1):
  print(image_batch.shape)
  print(labels.numpy())

(32, 224, 224, 3)
[1 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 1 1 0 1 0 1 1 1 0 0 1 1 1 1 1 1]


In [ ]:
def split_data(dataset , train_split =0.7 , val_split =0.3):
  dataset_size  =len(dataset)

  train_size = int(train_split*dataset_size)
  val_size = int(val_split*dataset_size)

  train_data = dataset.take(train_size)
  val_data =dataset.skip(train_size).take(val_size)

  return train_data ,val_data

In [ ]:
train_data  , val_data = split_data(dataset)

In [ ]:

len(train_data)

31

In [ ]:
len(val_data)

13

DATA AUGMENTATION

In [ ]:
resize_rescale = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
    tf.keras.layers.experimental.preprocessing.Resizing(224,224),
    
])

data_augmentation =tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.4)
])

# **MODEL**

In [ ]:
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense
from keras.models import Model
from keras.optimizers import Adam


In [ ]:
input_shape = (224 , 224 ,3)

base_model = tf.keras.applications.resnet50.ResNet50(weights = 'imagenet' ,include_top = False ,input_shape = input_shape)
base_model.trainable =False

inputs =tf.keras.Input(shape = input_shape)

x = resize_rescale(inputs)
x = data_augmentation(x)

x = base_model(x, training = False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(1024 , activation = 'relu')(x)
x= tf.keras.layers.Dense(1024, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

ResNet50Model = Model(inputs , outputs)

ResNet50Model.compile(
    optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy']
)


In [ ]:
ResNet50Model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d_1   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_1 (Dropout)         (None, 2048)              0         
                                                                 
 dense_3 (Dense)             (None, 1024)              2098176   
                                                           

In [ ]:
from keras.callbacks import ModelCheckpoint

callback_list = [ModelCheckpoint(filepath = '/content/drive/MyDrive/HACKMOL/Adult_ResNet50Model.h5' 
                                 ,monitor='val_accuracy' ,verbose = 1 , save_best_only = True )]

model = ResNet50Model.fit(train_data , epochs = 30 ,validation_data =val_data , callbacks=[callback_list])

Epoch 1/30
31/31 [==============================] - ETA: 0s - loss: 0.6115 - accuracy: 0.6996
Epoch 1: val_accuracy improved from -inf to 0.75721, saving model to /content/drive/MyDrive/HACKMOL/Adult_ResNet50Model.h5
31/31 [==============================] - 26s 615ms/step - loss: 0.6115 - accuracy: 0.6996 - val_loss: 0.5203 - val_accuracy: 0.7572
Epoch 2/30
31/31 [==============================] - ETA: 0s - loss: 0.5685 - accuracy: 0.7480
Epoch 2: val_accuracy improved from 0.75721 to 0.77404, saving model to /content/drive/MyDrive/HACKMOL/Adult_ResNet50Model.h5
31/31 [==============================] - 13s 403ms/step - loss: 0.5685 - accuracy: 0.7480 - val_loss: 0.5051 - val_accuracy: 0.7740
Epoch 3/30
31/31 [==============================] - ETA: 0s - loss: 0.5524 - accuracy: 0.7500
Epoch 3: val_accuracy improved from 0.77404 to 0.77885, saving model to /content/drive/MyDrive/HACKMOL/Adult_ResNet50Model.h5
31/31 [==============================] - 16s 501ms/step - loss: 0.5524 - accura